In [2]:
import torch
from torchvision import datasets, transforms,io
from torch.utils.data import DataLoader
import os


data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(0.2),
        transforms.RandomRotation(0.2),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

root_dir = ""
image_datasets = {
    x: datasets.ImageFolder(os.path.join(root_dir, x), data_transforms[x])
    for x in ['train', 'test','val']}
dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=32, shuffle=True)
    for x in ['train', 'test','val']}
dataset_sizes = {
    x: len(image_datasets[x]) for x in ['train', 'test','val']}

class_names = image_datasets['train'].classes

label_map = {class_name: i for i, class_name in enumerate(class_names)}

print(label_map)

{'Tomato Bacterial spot': 0, 'Tomato Early blight': 1, 'Tomato Healthy': 2, 'Tomato Late blight': 3, 'Tomato Leaf Mold': 4, 'Tomato Mosaic virus': 5, 'Tomato Septoria leaf spot': 6, 'Tomato Spider mites': 7, 'Tomato Target Spot': 8, 'Tomato Yellow Leaf Curl Virus': 9}


In [3]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x25db5c55850>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x25dbae76060>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x25d992ae9f0>}

In [5]:
import torch
import torch.nn as nn
from torchvision import models

model_ft = models.vgg19(weights='IMAGENET1K_V1')
num_ftrs = model_ft.classifier[6].in_features

for param in model_ft.parameters():
    param.requires_grad = False

model_ft.classifier[6] = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 10),
    nn.Softmax(dim=1)
)

optimizer = torch.optim.Adam(model_ft.parameters())
criterion = nn.CrossEntropyLoss()
metrics = ['accuracy']

print(model_ft)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [7]:
def calculate_accuracy(target, predicted):
    correct = (predicted == target).sum().item()
    return correct

def val(model, test_data, device):
    import numpy as np
    model.eval()
    not_correct = 0
    total_correct = 0
    with torch.no_grad():
        for batch,(data, target) in enumerate(test_data,start=1):
            data, target = data.to(device), target.to(device)
            pred = model(data)
            pred = torch.argmax(pred,dim=1)
            correct = calculate_accuracy(target, pred)
            total_correct += correct
            not_correct += 32-correct
            if batch==15:
                print(f"Accuracy: {np.round(total_correct/(total_correct+not_correct),3)} %  |   Loss: {np.round(not_correct/(total_correct+not_correct),3)}   | Correct: {total_correct} | Not Correct: {not_correct}")
                break


In [8]:
def training(model, train_data):
  model.train()
  model.to(device)
  for b_i,(data, target) in enumerate(train_data,start=1):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    y_pred = model(data)
    loss = criterion(y_pred, target)
    loss.backward()
    optimizer.step()
    if b_i%10==0:
      print(f"Batch = {b_i} | {b_i*len(data)}/{len(train_data.dataset)}   |   Loss = {loss.item()} | ",end=' ')
      batch_accuracy(target, y_pred)
    if b_i%100==0:
      print()
      print("Validation score")
      val(model, dataloaders['val'], device)
def batch_accuracy(target, y_pred):
    model_ft.eval()
    correct = (torch.argmax(y_pred, dim=1) == target).sum().item()
    total = len(target)
    accuracy = correct / total * 100
    print(f"Batch accuracy: {accuracy} %",'\n')


In [9]:
training(model_ft,dataloaders['train'])

Batch = 10 | 320/40960   |   Loss = 2.278470516204834 |  Batch accuracy: 21.875 %
Batch = 20 | 640/40960   |   Loss = 2.1442954540252686 |  Batch accuracy: 37.5 %
Batch = 30 | 960/40960   |   Loss = 2.1179232597351074 |  Batch accuracy: 34.375 %
Batch = 40 | 1280/40960   |   Loss = 1.9473668336868286 |  Batch accuracy: 56.25 %
Batch = 50 | 1600/40960   |   Loss = 2.0327260494232178 |  Batch accuracy: 43.75 %
Batch = 60 | 1920/40960   |   Loss = 2.047346830368042 |  Batch accuracy: 40.625 %
Batch = 70 | 2240/40960   |   Loss = 2.229649543762207 |  Batch accuracy: 21.875 %
Batch = 80 | 2560/40960   |   Loss = 1.9982174634933472 |  Batch accuracy: 50.0 %
Batch = 90 | 2880/40960   |   Loss = 1.835901141166687 |  Batch accuracy: 68.75 %
Batch = 100 | 3200/40960   |   Loss = 1.9472124576568604 |  Batch accuracy: 53.125 %

Validation score
Accuracy: 0.529 %  |   Loss: 0.471   | Correct: 254 | Not Correct: 226

Batch = 110 | 3520/40960   |   Loss = 1.9963104724884033 |  Batch accuracy: 50.0 %


In [10]:
def calculate_accuracy(target, predicted):
    correct = (predicted == target).sum().item()
    return correct

def test(model, test_data, device):
    import numpy as np
    model.eval()
    not_correct = 0
    total_correct = 0
    with torch.no_grad():
        for batch,(data, target) in enumerate(test_data,start=1):
            data, target = data.to(device), target.to(device)
            pred = model(data)
            pred = torch.argmax(pred,dim=1)
            correct = calculate_accuracy(target, pred)
            total_correct += correct
            not_correct += 32-correct
        print(f"Accuracy: {np.round(total_correct/(total_correct+not_correct),3)} %  |   Loss: {np.round(not_correct/(total_correct+not_correct),3)}   | Correct: {total_correct} | Not Correct: {not_correct}")


In [11]:
test(model_ft, dataloaders['test'], device)

Accuracy: 0.68 %  |   Loss: 0.32   | Correct: 6963 | Not Correct: 3277


In [13]:
test(model_ft, dataloaders['val'], device)

Accuracy: 0.671 %  |   Loss: 0.329   | Correct: 4124 | Not Correct: 2020


In [12]:
torch.save(model_ft.state_dict(), 'vgg16_model.h5')
print('Saved trained model as %s ' % 'vgg16_model.h5')

Saved trained model as vgg16_model.h5 
